# import packages

In [1]:
import pandas as pd
import numpy as np
import json
import torch
from torch import nn
import math

# Evironment Setup

In [2]:
import os

seed = 42
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: ", device)

#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

EXP_NAME = 'BERT_without_pretrainig'

Device:  cuda:1


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# import os
# os.chdir('/content/drive/MyDrive/NLP')

# preprocess data

In [5]:
with open('../data/english-train.json', 'r') as json_file:
    english_train = json.load(json_file)
with open('../data/english-dev.json', 'r') as json_file:
    english_dev = json.load(json_file)
with open('../data/english-test.json', 'r') as json_file:
    english_test = json.load(json_file)


In [6]:
english_dev[0]['description']

'good day. this morning i coughed for the very first time in a long time. with the corona virus around i feel the need to report this. i coughed for about 5 min. i have no fever, not tired and chest feels weird. what should i do?'

In [7]:
context_seq = []
patient_ques = []
docter_ans = []

context_seq.extend([i['description'] for i in english_train])
patient_ques.extend([i['utterances'][0] for i in english_train])
docter_ans.extend([i['utterances'][1] for i in english_train])

context_seq.extend([i['description'] for i in english_dev])
patient_ques.extend([i['utterances'][0] for i in english_dev])
docter_ans.extend([i['utterances'][1] for i in english_dev])

# Changed
# context_seq.extend([i['description'] for i in english_test])
# patient_ques.extend([i['utterances'][0] for i in english_test])
# docter_ans.extend([i['utterances'][1] for i in english_test])

context_seq = ['SOS '+i+' EOS' for i in context_seq]
patient_ques = ['SOS '+i[9:]+' EOS' for i in patient_ques]
docter_ans = ['SOS '+i[8:]+' EOS' for i in docter_ans]

tot_data = []
all_patient_ques = []
for i in range(len(patient_ques)):
  all_patient_ques.append(patient_ques[i]+" SEP "+context_seq[i])

tot_data.extend(all_patient_ques)
tot_data.extend(docter_ans)

In [8]:
# Changed
context_seq_test = []
patient_ques_test = []
docter_ans_test = []

context_seq_test.extend([i['description'] for i in english_test])
patient_ques_test.extend([i['utterances'][0] for i in english_test])
docter_ans_test.extend([i['utterances'][1] for i in english_test])

context_seq_test = ['SOS '+i+' EOS' for i in context_seq_test]
patient_ques_test = ['SOS '+i[9:]+' EOS' for i in patient_ques_test]
docter_ans_test = ['SOS '+i[8:]+' EOS' for i in docter_ans_test]

tot_data_test = []
all_patient_ques_test = []

for i in range(len(patient_ques_test)):
  all_patient_ques_test.append(patient_ques_test[i]+" SEP "+context_seq_test[i])
  
tot_data_test.extend(all_patient_ques_test)
tot_data_test.extend(docter_ans_test)

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
dialogs = tot_data
# Fit the tokenizer on the texts
# tokenizer.fit_on_texts(dialogs)
tokenizer.fit_on_texts(dialogs + tot_data_test) # Changed

# Convert texts to sequences of integers
all_patient_ques = tokenizer.texts_to_sequences(all_patient_ques)
patient_ques = tokenizer.texts_to_sequences(patient_ques)
docter_ans = tokenizer.texts_to_sequences(docter_ans)

# Pad the sequences to have equal length
# patient_ques = pad_sequences(patient_ques, padding='post')
# docter_ans = pad_sequences(docter_ans, padding='post')

print("Word Index = " , tokenizer.word_index)
print("Sequences = " , patient_ques)
# print("Padded Sequences:")
# print(padded_sequences)

2023-12-07 11:09:51.303442: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-07 11:09:51.369661: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 11:09:51.369722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 11:09:51.371533: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 11:09:51.382406: I tensorflow/core/platform/cpu_feature_guar

Word Index =  {'<OOV>': 1, 'i': 2, 'and': 3, 'to': 4, 'sos': 5, 'eos': 6, 'the': 7, 'a': 8, 'is': 9, 'have': 10, 'of': 11, 'for': 12, 'in': 13, 'you': 14, 'with': 15, 'or': 16, 'my': 17, 'if': 18, 'your': 19, 'be': 20, 'covid': 21, 'sep': 22, 'it': 23, 'not': 24, '19': 25, 'pneumonia': 26, 'can': 27, 'on': 28, 'are': 29, 'symptoms': 30, 'that': 31, 'this': 32, 'me': 33, 'like': 34, 'but': 35, 'should': 36, 'get': 37, 'from': 38, 'as': 39, 'cough': 40, 'would': 41, 'do': 42, 'been': 43, 'fever': 44, 'throat': 45, 'am': 46, 'was': 47, 'at': 48, 'has': 49, 'will': 50, 'what': 51, 'had': 52, 'no': 53, 'coronavirus': 54, 'so': 55, 'video': 56, 'virus': 57, 'brief': 58, 'tested': 59, 'days': 60, 'now': 61, 'chat': 62, 'text': 63, 'sore': 64, 'he': 65, 'chest': 66, 'doctor': 67, 'she': 68, 'also': 69, 'any': 70, 'need': 71, 'test': 72, 'contact': 73, 'infection': 74, 'home': 75, 'there': 76, 'treatment': 77, 'more': 78, 'an': 79, 'take': 80, 'about': 81, 'last': 82, 'her': 83, 'may': 84, 'by'

In [11]:
# Changed
all_patient_ques_test = tokenizer.texts_to_sequences(all_patient_ques_test)
patient_ques_test = tokenizer.texts_to_sequences(patient_ques_test)
docter_ans_test = tokenizer.texts_to_sequences(docter_ans_test)

print("Sequences = " , patient_ques_test)

Sequences =  [[5, 2, 10, 90, 7, 30, 656, 44, 2, 269, 4, 3248, 3, 123, 373, 2, 27, 37, 59, 18, 2, 288, 4, 212, 24, 213, 18, 2, 36, 68, 446, 33, 119, 3249, 3250, 1752, 68, 373, 2, 27, 80, 23, 18, 2, 135, 226, 212, 258, 23, 50, 303, 384, 371, 318, 6], [5, 50, 17, 257, 38, 3251, 3252, 1837, 816, 15, 1117, 414, 33, 38, 86, 57, 576, 15, 1837, 2232, 6], [5, 380, 3253, 166, 38, 3254, 4, 2161, 1043, 3255, 49, 52, 44, 11, 744, 418, 48, 206, 3, 108, 115, 12, 3256, 59, 840, 12, 122, 3, 170, 2239, 11, 275, 340, 449, 54, 3257, 6], [5, 17, 25, 178, 128, 289, 47, 179, 15, 26, 194, 331, 166, 3, 47, 244, 119, 35, 15, 884, 652, 1317, 65, 148, 2363, 38, 7, 260, 30, 35, 107, 65, 9, 13, 7, 5302, 65, 2674, 4, 94, 139, 4, 7, 67, 12, 1558, 11, 5303, 153, 40, 9, 148, 700, 3, 153, 1698, 9, 191, 29, 76, 70, 223, 2264, 3258, 12, 153, 275, 18, 65, 2980, 336, 77, 6], [5, 98, 76, 33, 3, 17, 3259, 128, 10, 122, 34, 30, 3, 10, 43, 13, 73, 15, 136, 31, 795, 38, 456, 2267, 15, 79, 3260, 582, 122, 36, 2, 37, 59, 12, 21, 2

In [12]:
segmented_embedding = []
sep_token_id = tokenizer.word_index['sep']
for sequence in all_patient_ques:
    segment_ids = []
    current_segment_id = 1
    for token_id in sequence:
        segment_ids.append(current_segment_id)
        if token_id == sep_token_id:
            current_segment_id = 2
    segmented_embedding.append(segment_ids)

In [13]:
# Changed
segmented_embedding_test = []
sep_token_id = tokenizer.word_index['sep']
for sequence in all_patient_ques_test:
    segment_ids = []
    current_segment_id = 1
    for token_id in sequence:
        segment_ids.append(current_segment_id)
        if token_id == sep_token_id:
            current_segment_id = 2
    segmented_embedding_test.append(segment_ids)

# Initialize Dataloader

In [14]:
# Create train dataset and dataloader

from torch.utils.data import DataLoader
from src.Datasets import ConversationDataset

batch_size = 32  # Define the batch size
pat_qus = torch.tensor(pad_sequences(all_patient_ques, padding='post'))
doc_ans = torch.tensor(pad_sequences(docter_ans, padding='post'))
seg_emb = torch.tensor(pad_sequences(segmented_embedding, padding='post'))

conversation_dataset = ConversationDataset(pat_qus, doc_ans, seg_emb)
conversation_dataloader = DataLoader(conversation_dataset, batch_size=batch_size, shuffle=True)

In [15]:
# Create test dataset and dataloader # Changed
pat_qus_test = torch.tensor(pad_sequences(all_patient_ques_test, padding='post'))
doc_ans_test = torch.tensor(pad_sequences(docter_ans_test, padding='post'))
seg_emb_test = torch.tensor(pad_sequences(segmented_embedding_test, padding='post'))
conversation_dataset_test = ConversationDataset(pat_qus_test, doc_ans_test, seg_emb_test)
conversation_dataloader_test = DataLoader(conversation_dataset_test, batch_size=batch_size, shuffle=False)

# Initialize BERT

In [16]:
# Initialize the model
 
from src.BERT import BERT

model = BERT(num_embeddings=len(tokenizer.word_index)+10).to(device)

for i, data in enumerate(conversation_dataloader):
    patient_questions = data['patient_question'].to(device)
    doctor_answers = data['doctor_answer'].to(device)
    segment_embeddings = data['segment_embedding'].to(device)
    break   # Changed

print(patient_questions.shape)
print(doctor_answers.shape)
print(segment_embeddings.shape)

output = model(patient_questions, segment_embeddings)[:,201:,:]

output.shape

torch.Size([32, 413])
torch.Size([32, 212])
torch.Size([32, 413])


torch.Size([32, 212, 5447])

# Define Loss and Optimizer

In [17]:
# Define loss function and optimizer

from torch.nn import CrossEntropyLoss
from torch.optim import AdamW

loss_function = CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=0.001)


In [18]:
def array_to_str(arr, vocab):
    return " ".join(vocab[a] for a in arr)
    
    
def make_generation_text(inp, pred):
    outputs = u""
    for i in range(len(inp)):
        outputs += u"Input | Output #{}: {} | {}\n".format(i, inp[i], pred[i])
    return outputs

# Train

In [19]:
import torch
from tqdm import tqdm
import pandas as pd


train_losses, test_losses = [], []

# Training loop
num_epochs = 100
for epoch in range(num_epochs):  # num_epochs should be defined
    model.train()
    total_loss = 0
    total_loss_test = 0   # Changed

    # Progress Bar
    batch_bar   = tqdm(total=len(conversation_dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5)

    for i, data in enumerate(conversation_dataloader):
        # Forward pass
        
        model.train()

        patient_questions = data['patient_question'].to(device)
        doctor_answers = data['doctor_answer'].to(device).long()
        segment_embeddings = data['segment_embedding'].to(device)

        # if(len(patient_questions)!=batch_size):
        #   print('jump')
        #   continue
        optimizer.zero_grad()
        output = model(patient_questions, segment_embeddings)

        # output = output[:, 201:, :] # Changed
        ans_seq_len = doctor_answers.shape[1]
        output = output[:, :ans_seq_len, :] # Changed
        # output = output.softmax(dim=-1)
        # Compute loss - ensure doctor_answers is the correct target and has the right shape

        loss = loss_function(output.reshape(-1, output.size(-1)), doctor_answers.reshape(-1))
        total_loss += loss.item()
    

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        batch_bar.set_postfix(
            loss        = "{:.04f}".format(float(total_loss / ((i+1) * conversation_dataloader.batch_size))),
        )

        batch_bar.update() # Update tqdm bar

    batch_bar.close()
    
    loss = total_loss/len(conversation_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss}")
    train_losses.append(loss)

    # Progress Bar
    batch_bar   = tqdm(total=len(conversation_dataloader_test), dynamic_ncols=True, leave=False, position=0, desc='Test', ncols=5)
    
    # Changed
    with torch.no_grad():
        for i, data in enumerate(conversation_dataloader_test):
            # Forward pass
            
            model.eval()

            patient_questions = data['patient_question'].to(device)
            doctor_answers = data['doctor_answer'].to(device).long()
            segment_embeddings = data['segment_embedding'].to(device)

            output = model(patient_questions, segment_embeddings)

            ans_seq_len = doctor_answers.shape[1]
            output = output[:, :ans_seq_len, :] # Changed

            loss = loss_function(output.reshape(-1, output.size(-1)), doctor_answers.reshape(-1))
            total_loss_test += loss.item()

            batch_bar.set_postfix(
                loss        = "{:.04f}".format(float(total_loss / ((i+1) * conversation_dataloader_test.batch_size))),
            )

            batch_bar.update() # Update tqdm bar

        if epoch % 10 == 0 or epoch == num_epochs-1:
            
            pred = torch.argmax(output, dim=-1)

            temp = [seq[:seq.index(0)] if seq.count(0) != 0 else seq for seq in patient_questions.cpu().tolist()]
            print("Patient      :", tokenizer.sequences_to_texts(temp)[:1])
            temp = [seq[:seq.index(0)] if seq.count(0) != 0 else seq for seq in doctor_answers.cpu().tolist()]
            print("Doctor(targ) :", tokenizer.sequences_to_texts(temp)[:1])
            temp = [seq[:seq.index(0)] if seq.count(0) != 0 else seq for seq in pred.cpu().tolist()]
            print("Doctor(pred) :", tokenizer.sequences_to_texts(temp)[:1])
            print("Loss : ", loss_function(output[0].reshape(-1, output.size(-1)), doctor_answers[0].reshape(-1)))

            inp = [seq[:seq.index(0)] if seq.count(0) != 0 else seq for seq in patient_questions.cpu().tolist()]
            inp = tokenizer.sequences_to_texts(inp)
            temp = []
            for text in inp:
                start_index = text.index("sep")
                temp.append(text[:start_index-1])
            inp = temp
            generated = [seq[:seq.index(0)] if seq.count(0) != 0 else seq for seq in pred.cpu().tolist()]
            generated = tokenizer.sequences_to_texts(generated)

            
            with open(os.path.join('../experiments', EXP_NAME, 'generated-texts-{}-test.txt'.format(epoch)), 'w') as fw:
                fw.write(make_generation_text(inp, generated))

    batch_bar.close()

    loss = total_loss_test/len(conversation_dataloader_test)
    print(f"Epoch {epoch+1}/{num_epochs}, Test  Loss: {loss}")
    test_losses.append(loss)
    print("===============================================")

Train:   6%|▌         | 1/17 [00:00<00:03,  4.51it/s, loss=0.2846]

Epoch 1/100, Train Loss: 2.862954048549428


Patient      : ['sos hi i’ve had a cold since 13 march i have since started coughing and just don’t seem to be recovering one day i will be feeling fine and the next day back to feeling terrible lots of tight phlegm and very sore throat eos sep sos hi i’ve had a cold since 13 march i have since started coughing and just don’t seem to be recovering one day i will be feeling fine and the next day back to feeling terrible lots of tight phlegm and very sore throat eos']
Doctor(targ) : ['sos in brief too soon to recover on aveage viral respiratory illnesses may take 2 3 weeks for full recovery and severe cases may take 4 6 weeks if your symptoms worsen you should contact your health care provider to inquire regarding corona virus testing or quarantine would you like to video or text chat with me eos']
Doctor(pred) : ['']
Loss :  tensor(3.3599, device='cuda:1')
Epoch 1/100, Test  Loss: 3.934788703918457


Epoch 2/100, Train Loss: 2.1044073315227734


Epoch 2/100, Test  Loss: 3.868319034576416


...

Epoch 98/100, Test  Loss: 8.060383796691895


Epoch 99/100, Train Loss: 0.020941137390978196


Epoch 99/100, Test  Loss: 7.987030267715454


Epoch 100/100, Train Loss: 0.019097301150288654


Patient      : ['sos hi i’ve had a cold since 13 march i have since started coughing and just don’t seem to be recovering one day i will be feeling fine and the next day back to feeling terrible lots of tight phlegm and very sore throat eos sep sos hi i’ve had a cold since 13 march i have since started coughing and just don’t seem to be recovering one day i will be feeling fine and the next day back to feeling terrible lots of tight phlegm and very sore throat eos']
Doctor(targ) : ['sos in brief too soon to recover on aveage viral respiratory illnesses may take 2 3 weeks for full recovery and severe cases may take 4 6 weeks if your symptoms worsen you should contact your health care provider to inquire regarding corona virus testing or quarantine would you like to video or text chat with me eos']
Doctor(pred) : ['sos sos unlikely be be being for for from more www to to fever for care then may sick monitor eos 19 pains worse or your or if with cough with enough']
Loss :  tensor(6.5404, 

In [9]:
import openai
import os

# Fill the run id and epoch number to be used for submission.
# You will use the same run id and epoch number to generate the handin.
submission_epoch = 99 # TODO

n_tests = 29

with open(os.path.join('../experiments', EXP_NAME, 'generated-texts-{}-test.txt'.format(submission_epoch)), 'r', encoding='utf-8') as f:
    generated = list(f)

assert len(generated) == n_tests
for item in generated:
    assert type(item) is str

parsed_generated = []

for text in generated:
    start_index = text.index("sos")
    temp = text[start_index+4:]
    generation_start_index = temp.index("| ")
    parsed_text = temp[:generation_start_index] + temp[generation_start_index+2:]
    parsed_text = parsed_text.replace("<eol>", "\n")
    parsed_generated.append(parsed_text)

def perplexity(text, modelname):
    """Compute the perplexity of the provided text."""
    completion = openai.Completion.create(
        model=modelname,
        prompt=text,
        logprobs=0,
        max_tokens=0,
        temperature=1.0,
        echo=True)
    token_logprobs = completion['choices'][0]['logprobs']['token_logprobs']
    ll = np.mean([i for i in token_logprobs if i is not None])
    ppl = np.exp(-ll)
    return ppl

# Add you API key here to get perplexity. However, delete the key from the notebook before creating the handin.
# REMEMBER: ALWAYS KEEP YOUR API KEYS AND SECRETS SECURE.
with open('/mnt/sda/jaeheon/jaeheon/CMU/IDL/openai_key', 'r') as f:
    openai_key = f.read()
    f.close()
openai.api_key = openai_key # TODO
modelname = 'text-embedding-ada-002'

perps = [perplexity(text, modelname) for text in tqdm(parsed_generated)]
avg_perp = np.mean(perps)

# Report this number when running the makefile to create the handin
print("Your mean perplexity for generated sequences: {}".format(avg_perp))


perplexity = [None for _ in range(num_epochs - 1)].append(avg_perp)
df_losses = pd.DataFrame({'train_loss':train_losses, 'test_loss':test_losses, 'perplexity':perplexity})
df_losses.to_csv(os.path.join('experiments', EXP_NAME, 'loss.csv'))

100%|██████████| 29/29 [00:09<00:00,  2.93it/s]

Your mean perplexity for generated sequences: 7552.422608204207


In [ ]:
loss_function(output.reshape(-1, output.size(-1)), doctor_answers.reshape(-1))